The core building block of LangChain applications is the LLMChain. This combines three things:

LLM: The language model is the core reasoning engine here. In order to work with LangChain, you need to understand the different types of language models and how to work with them.

Prompt Templates: This provides instructions to the language model. This controls what the language model outputs, so understanding how to construct prompts and different prompting strategies is crucial.

Output Parsers: These translate the raw response from the LLM to a more workable format, making it easy to use the output downstream.

1.LLMs

There are two types of language models, which in LangChain are called:

LLMs: this is a language model which takes a string as input and returns a string

ChatModels: this is a language model which takes a list of messages as input and returns a message

In [2]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv, find_dotenv
import openai
import os
load_dotenv(find_dotenv())
openai.api_key = os.environ["OPENAI_API_KEY"]

llm = OpenAI()
chat_model = ChatOpenAI()

llm.predict("hi!")

'\\\\hi!");\n    var0.endEqualityExpr(10);\n    var0.endXPath();\n    var0.startProcessingInstructionNodeStep(10, "(0) startAbsoluteLocationPath()\\n(1) endAndExpr(true)\\n(2) startUnionExpr()\\n");\n    var0.endFunction();\n    var0.startRelativeLocationPath();\n    var0.startProcessingInstructionNodeStep(100, "(0) startAbsoluteLocationPath()\\n(1) endAndExpr(true)\\n\\n(0) startAbsoluteLocationPath()\\n(1) endUnaryExpr(10)\\n(2) number(1)\\n(3) startAllNodeStep(-1)\\n(4) startOrExpr()\\n(5) endUnaryExpr(0)\\n(6) endFunction()\\n\\n          ^");\n    var0.startNameStep(1, "com.werken.saxpath.XPathSyntaxException: 0: (0) startAbsoluteLocationPath()\\n(1) endAndExpr'

In [3]:
chat_model.predict("hi!")

'Hello! How can I assist you today?'

The OpenAI and ChatOpenAI objects are basically just configuration objects. You can initialize them with parameters like temperature and others, and pass them around.

In [4]:
# let's use the predict method to run over a string input.

text = "What would be a good company name for a company that makes colorful socks?"

llm.predict(text)

'\n\nBrightToe Socks.'

In [5]:
chat_model.predict(text)

'Rainbow Threads'

2.Prompt templates

Most LLM applications do not pass user input directly into an LLM. Usually they will add the user input to a larger piece of text, called a prompt template
PromptTemplates help with exactly this! They bundle up all the logic for going from user input into a fully formatted prompt

In [7]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("What is a good name for a company that makes {product}?")
prompt.format(product="colorful socks")

'What is a good name for a company that makes colorful socks?'

In [8]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

chat_prompt.format_messages(input_language="English", output_language="French", text="I love programming.")

[SystemMessage(content='You are a helpful assistant that translates English to French.', additional_kwargs={}),
 HumanMessage(content='I love programming.', additional_kwargs={}, example=False)]

3.Output parsers

OutputParsers convert the raw output of an LLM into a format that can be used downstream. There are few main type of OutputParsers, including:

1.Convert text from LLM -> structured information (eg JSON)

2.Convert a ChatMessage into just a string

3.Convert the extra information returned from a call besides the message (like OpenAI function invocation) into a string.

In [10]:
from langchain.schema import BaseOutputParser

class CommaSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""


    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return text.strip().split(", ")

CommaSeparatedListOutputParser().parse("hi, bye")

['hi', 'bye']

LLMChain

We can now combine all these into one chain. This chain will take input variables, pass those to a prompt template to create a prompt, pass the prompt to an LLM, and then pass the output through an (optional) output parser. This is a convenient way to bundle up a modular piece of logic.

In [12]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
from langchain.schema import BaseOutputParser

class CommaSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""


    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return text.strip().split(", ")

template = """You are a helpful assistant who generates comma separated lists.
A user will pass in a category, and you should generate 5 objects in that category in a comma separated list.
ONLY return a comma separated list, and nothing more."""
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chain = LLMChain(
    llm=ChatOpenAI(),
    prompt=chat_prompt,
    output_parser=CommaSeparatedListOutputParser()
)
chain.run("colors")

['red', 'blue', 'green', 'yellow', 'orange']